# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:100]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint

In [14]:
save_path = 'ckpt'

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

## Training

http://localhost:6006

In [16]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs, 
    callbacks=[checkpoint]
)

W0920 13:49:33.031279 22920 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\math_grad.py:1394: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
2/3 [===================>..........] - ETA: 0s - loss: 2.3638 - accuracy: 0.0278    
Epoch 00001: val_accuracy improved from -inf to 0.07292, saving model to ckpt


W0920 13:49:34.531265 22920 deprecation.py:506] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


3/3 [==============================] - 2s 661ms/step - loss: 2.3902 - accuracy: 0.0294 - val_loss: 2.3492 - val_accuracy: 0.0729
Epoch 2/5
2/3 [===================>..........] - ETA: 0s - loss: 2.3568 - accuracy: 0.0938
Epoch 00002: val_accuracy improved from 0.07292 to 0.17708, saving model to ckpt
3/3 [==============================] - 2s 608ms/step - loss: 2.3123 - accuracy: 0.1562 - val_loss: 2.2900 - val_accuracy: 0.1771
Epoch 3/5
1/3 [=========>....................] - ETA: 0s - loss: 2.3461 - accuracy: 0.0625
Epoch 00003: val_accuracy did not improve from 0.17708
3/3 [==============================] - 1s 171ms/step - loss: 2.3539 - accuracy: 0.1471 - val_loss: 2.2858 - val_accuracy: 0.1771
Epoch 4/5
2/3 [===================>..........] - ETA: 0s - loss: 2.3473 - accuracy: 0.0833
Epoch 00004: val_accuracy improved from 0.17708 to 0.19792, saving model to ckpt
3/3 [==============================] - 2s 629ms/step - loss: 2.3158 - accuracy: 0.1471 - val_loss: 2.2940 - val_accuracy: 0

# Saving Model

In [11]:
save_path = 'my_model.h5'

In [12]:
model.save(save_path, include_optimizer=True)

In [13]:
model = tf.keras.models.load_model('my_model.h5')

UnicodeEncodeError: 'mbcs' codec can't encode characters in position 0--1: invalid character

# Saving Model - 2

In [14]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [15]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [16]:
import h5py

model_file = h5py.File('my_model.h5','r+')    

In [17]:
model_file.keys()

<KeysViewHDF5 ['model_weights']>

In [18]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [19]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [20]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [21]:
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

array([[[[ 0.09818335, -0.040579  ,  0.13305335,  0.02204266,
          -0.10216027,  0.02127637,  0.09995307, -0.05189637,
          -0.02596031, -0.1208975 ,  0.07827646, -0.11074499,
          -0.12479363,  0.13374685, -0.12778614,  0.13258083,
           0.04740551, -0.04140083, -0.04770781, -0.02668962,
           0.08223413, -0.12994385, -0.05800721,  0.06918089,
          -0.03416525, -0.02452487, -0.05135973, -0.09845071,
           0.08185707, -0.05858433, -0.10869929,  0.04106784],
         [ 0.04933999, -0.01399218,  0.11830284, -0.08382471,
           0.12142013,  0.09820217,  0.04161103,  0.06070307,
           0.05761907, -0.05570982, -0.13591154,  0.0451691 ,
          -0.02922563,  0.03949469,  0.01660302, -0.12025861,
           0.03508267, -0.12586522, -0.10682852,  0.08048049,
          -0.00671434,  0.02743231,  0.04287182,  0.05602297,
           0.09651412, -0.11455081, -0.03970913,  0.08595033,
           0.05848742, -0.05228804,  0.11407588, -0.12296423],
      